<a href="https://colab.research.google.com/github/pmxfa/sp-shapely/blob/main/sp_timegan_electricity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install synthcity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.2/552.2 kB 42.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchtext to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is sti

# Training

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
import warnings
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import synthcity.logger as log
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import TimeSeriesDataLoader

log.add(sink=sys.stderr, level="INFO")


Mounted at /content/drive
[KeOps] Compiling cuda jit compiler engine ... OK
[pyKeOps] Compiling nvrtc binder for python ... OK


In [18]:
file_path = "/content/drive/Shareddrives/sp_env/datasets/Electricity Transformer Dataset (ETDataset)/ETTh1.csv"

df = pd.read_csv(file_path)
print(df.head())
print(df.info())
print(df.isnull().sum())

                  date   HUFL   HULL   MUFL   MULL   LUFL   LULL         OT
0  2016-07-01 00:00:00  5.827  2.009  1.599  0.462  4.203  1.340  30.531000
1  2016-07-01 01:00:00  5.693  2.076  1.492  0.426  4.142  1.371  27.787001
2  2016-07-01 02:00:00  5.157  1.741  1.279  0.355  3.777  1.218  27.787001
3  2016-07-01 03:00:00  5.090  1.942  1.279  0.391  3.807  1.279  25.044001
4  2016-07-01 04:00:00  5.358  1.942  1.492  0.462  3.868  1.279  21.948000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17420 entries, 0 to 17419
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    17420 non-null  object 
 1   HUFL    17420 non-null  float64
 2   HULL    17420 non-null  float64
 3   MUFL    17420 non-null  float64
 4   MULL    17420 non-null  float64
 5   LUFL    17420 non-null  float64
 6   LULL    17420 non-null  float64
 7   OT      17420 non-null  float64
dtypes: float64(7), object(1)
memory usage: 1.1+ MB
None
date    0

In [19]:
# Convert 'date' to datetime, set as index, and sort
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.sort_index(inplace=True)

# Keep the latest 5000 rows
df_latest = df.tail(5000)

# Train-test split: 70% for training (for TimeGAN), 30% for testing (TSTR)
train_size = int(0.7 * len(df_latest))
df_train = df_latest.iloc[:train_size]
df_test = df_latest.iloc[train_size:]  # use later for LSTM-TSTR

# Normalize the data using StandardScaler
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(df_train)
df_scaled_train = pd.DataFrame(scaled_train, columns=df_train.columns, index=df_train.index)
scaled_test = scaler.transform(df_test)
df_scaled_test = pd.DataFrame(scaled_train, columns=df_train.columns, index=df_train.index)

# Sequence length for time-series data (dataset = hourly; 24 hours)
sequence_length = 24
temporal_data = []
observation_times = []

# Generate sequences from df_scaled_train only
for start in range(len(df_scaled_train) - sequence_length + 1):
    sequence = df_scaled_train.iloc[start:start + sequence_length].reset_index(drop=True)
    temporal_data.append(sequence)
    observation_times.append(list(range(sequence_length)))  # relative time within the window

# Dummy outcome for TimeGAN (can be used in DataLoader)
dummy_outcome = pd.DataFrame(np.zeros(len(temporal_data)), columns=["outcome"])

# Create DataLoader for TimeGAN
loader = TimeSeriesDataLoader(
    temporal_data=temporal_data,
    observation_times=observation_times,
    static_data=None,
    outcome=dummy_outcome,
)

# Print the loader info
print(f"TimeSeriesDataLoader created with {len(temporal_data)} sequences")


TimeSeriesDataLoader created with 3477 sequences


In [20]:
print(len(df_train))
print(loader.dataframe())

3500
       seq_id  seq_time_id  seq_temporal_HUFL  seq_temporal_HULL  \
0           0            0           0.642388           0.302720   
1           0            1           0.613277           0.410782   
2           0            2           0.656955           0.427003   
3           0            3           0.647244           0.378339   
4           0            4           0.711963           0.416189   
...       ...          ...                ...                ...   
83443    3476           19           0.841426           0.362118   
83444    3476           20           0.865681           0.362118   
83445    3476           21           0.864062           0.437818   
83446    3476           22           0.930425           0.481075   
83447    3476           23           0.791250           0.405375   

       seq_temporal_LUFL  seq_temporal_LULL  seq_temporal_MUFL  \
0               0.429806           0.589646           0.659849   
1               0.429806           0.666667   

In [23]:
hparams = {
          "mode": "LSTM", # default mode = RNN
}

# Load TimeGAN with custom parameters
syn_model = Plugins().get("timegan", **hparams)

[2025-04-26T08:38:53.152878+0000][1780][CRITICAL] module disabled: /usr/local/lib/python3.11/dist-packages/synthcity/plugins/generic/plugin_goggle.py
[2025-04-26T08:38:53.152878+0000][1780][CRITICAL] module disabled: /usr/local/lib/python3.11/dist-packages/synthcity/plugins/generic/plugin_goggle.py
[2025-04-26T08:38:53.156663+0000][1780][INFO] TimeGAN: mode = LSTM dataloader_sampling_strategy = imbalanced_time_censoring


In [26]:
# Print all parameters of initialized model
for attr in dir(syn_model):
    if not attr.startswith("_") and not callable(getattr(syn_model, attr)):
        print(f"{attr}: {getattr(syn_model, attr)}")

batch_size: 64
class_name: TimeGANPlugin
clipping_value: 0
compress_dataset: False
dataloader_sampling_strategy: imbalanced_time_censoring
device: cuda
discriminator_batch_norm: False
discriminator_dropout: 0.1
discriminator_loss: None
discriminator_lr: 0.001
discriminator_n_iter: 1
discriminator_n_layers_hidden: 3
discriminator_n_units_hidden: 300
discriminator_nonlin: leaky_relu
discriminator_weight_decay: 0.001
embedding_penalty: 10
encoder: None
encoder_max_clusters: 20
expecting_conditional: False
fitted: False
gamma_penalty: 1
generator_batch_norm: False
generator_dropout: 0.01
generator_loss: None
generator_lr: 0.001
generator_n_layers_hidden: 2
generator_n_units_hidden: 150
generator_nonlin: leaky_relu
generator_nonlin_out_continuous: tanh
generator_nonlin_out_discrete: softmax
generator_residual: True
generator_weight_decay: 0.001
mode: LSTM
module_name: synthcity.plugins.time_series.plugin_timegan
module_relative_path: ../time_series/plugin_timegan.py
moments_penalty: 100
n_i

## fitting the model

In [27]:
#  Train the model
syn_model.fit(loader)

[2025-04-26T08:44:30.262587+0000][1780][INFO] Encoding HUFL 5716080747266006699
[2025-04-26T08:44:39.400403+0000][1780][INFO] Encoding HULL 13614765943025521164
[2025-04-26T08:44:49.491653+0000][1780][INFO] Encoding LUFL 1878722292843447936
[2025-04-26T08:44:59.706269+0000][1780][INFO] Encoding LULL 1441795387440754216
[2025-04-26T08:45:10.185601+0000][1780][INFO] Encoding MUFL 16679051611867554908
[2025-04-26T08:45:20.384774+0000][1780][INFO] Encoding MULL 12097090215823483204
[2025-04-26T08:45:30.490830+0000][1780][INFO] Encoding OT 14455165812040384062
100%|██████████| 1000/1000 [3:18:18<00:00, 11.90s/it]
[2025-04-26T12:04:27.520656+0000][1780][INFO] Encoding outcome 17036877706523873670
[2025-04-26T12:04:28.262090+0000][1780][INFO] Epoch:0| train loss: 0.0, validation loss: 0.0
[2025-04-26T12:04:32.861423+0000][1780][INFO] Epoch:10| train loss: 0.0, validation loss: 0.0
[2025-04-26T12:04:36.949823+0000][1780][INFO] Epoch:20| train loss: 0.0, validation loss: 0.0
[2025-04-26T12:04:4

In [34]:
saved_model = syn_model.save()

In [3]:
from synthcity.utils.serialization import save_to_file, load_from_file

# Save model to drive
save_to_file('./adsgan_10_epochs.pkl', syn_model)

# Load the model
syn_model = load_from_file('/content/drive/Shareddrives/sp_env/synmodel.pkl')

In [29]:
#  Generate Synthetic Data
n_samples = len(temporal_data)
syn_data = syn_model.generate()
print(syn_data.shape)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.14 GiB. GPU 0 has a total capacity of 14.74 GiB of which 478.12 MiB is free. Process 17335 has 14.27 GiB memory in use. Of the allocated memory 11.83 GiB is allocated by PyTorch, and 2.29 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [30]:
syn_data_2 = syn_model.generate(count=n_samples)
print(syn_data_2.shape)

[2025-04-26T12:09:29.187807+0000][1780][INFO] [seq_time_id] quality loss for constraints ge = 0. Remaining 59117. prev length 83448. Original dtype int64.


KeyboardInterrupt: 

In [ ]:
# Save with automated format
import datetime
import os
# Get the current date and time
now = datetime.datetime.now()
timestamp = now.strftime("%m%d%y-%H%M%S")  # MMDDYY-HHMMSS format

# Define the base directory
base_dir = "/content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/electricity"  #CHANGE THIS
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Construct the filename
model_name = type(syn_model).__name__.lower() # Get model name dynamically
filename = f"{timestamp}-{model_name}-n_3000.csv"
filepath = os.path.join(base_dir, filename)

# Save the data
df_syn = syn_data.dataframe()
df_syn.to_csv(filepath, index=False)

print(f"Synthetic data saved to: {filepath}")

In [ ]:
# save model here

# Evaluation

## Prerequisites

In [ ]:
# Define selected columns explicitly
selected_columns = ['seq_temporal_HUFL', 'seq_temporal_HULL', 'seq_temporal_LUFL', 'seq_temporal_LULL', 'seq_temporal_MUFL', 'seq_temporal_MULL', 'seq_temporal_OT']

# Ensure real_data and synthetic_data only contain the selected columns
real_data = loader.dataframe()[selected_columns].to_numpy()
synthetic_data = syn_data.dataframe()[selected_columns].to_numpy()

In [ ]:
#  Check datasets

print(real_data, "\n ------------------------------------------------------- \n", synthetic_data)
print(type(real_data),type(synthetic_data))
print(real_data.shape,synthetic_data.shape)

""" TODO
[] add adjusting off dataset to fit min length here
[] remove min length stuff in helper funcs
"""

## Generate distance metrics

### Helper Functions

In [ ]:
from scipy.stats import wasserstein_distance, entropy
import numpy as np

def compute_wasserstein(real_data, synthetic_data, selected_columns):
    """
    Computes Wasserstein Distance between real and synthetic time-series data.

    """

    # Ensure both datasets have the same number of samples
    min_length = min(len(real_data), len(synthetic_data))
    real_trimmed = real_data[:min_length]  # Keep original order (no random sampling)
    synthetic_trimmed = synthetic_data[:min_length]  # Match size
    print(real_trimmed.shape,synthetic_trimmed.shape)

    wasserstein_results = {}

    # Compute Wasserstein Distance for each feature
    for i, col in enumerate(selected_columns):
        w_dist = wasserstein_distance(real_trimmed[:, i], synthetic_trimmed[:, i])
        wasserstein_results[col] = w_dist
        print(f"{w_dist}")

    return wasserstein_results

def compute_kl_divergence(real_data, synthetic_data, selected_columns, bins=50):
    """
    Computes KL Divergence between real and synthetic time-series data.

    """

    # Ensure both datasets have the same number of samples
    min_length = min(len(real_data), len(synthetic_data))
    real_trimmed = real_data[:min_length]  # Keep original order
    synthetic_trimmed = synthetic_data[:min_length]  # Match size

    kl_results = {}

    for i, col in enumerate(selected_columns):
        # Compute histogram-based probability distributions
        real_hist, _ = np.histogram(real_trimmed[:, i], bins=bins, density=True)
        synth_hist, _ = np.histogram(synthetic_trimmed[:, i], bins=bins, density=True)

        # Avoid zero probabilities (KL Divergence is undefined for zero values)
        real_hist += 1e-10
        synth_hist += 1e-10

        # Compute KL Divergence
        kl_div = entropy(real_hist, synth_hist)
        kl_results[col] = kl_div
        print(f"{kl_div}")

    return kl_results

### Generate Metrics

In [ ]:
# Compute Wasserstein Distance
wasserstein_results = compute_wasserstein(real_data, synthetic_data, selected_columns)
print("Wasserstein Distance Results:")
print(wasserstein_results)

# Compute KL Divergence
kl_results = compute_kl_divergence(real_data, synthetic_data, selected_columns)
print("KL Divergence Results:")
print(kl_results)

(32986, 7) (32986, 7)
0.6333693243548061
0.5754466980076927
0.29508214647631575
0.3802229781894522
0.7765970668651524
0.6079451106316505
0.5188429106552913
Wasserstein Distance Results:
{'seq_temporal_HUFL': 0.6333693243548061, 'seq_temporal_HULL': 0.5754466980076927, 'seq_temporal_LUFL': 0.29508214647631575, 'seq_temporal_LULL': 0.3802229781894522, 'seq_temporal_MUFL': 0.7765970668651524, 'seq_temporal_MULL': 0.6079451106316505, 'seq_temporal_OT': 0.5188429106552913}
11.15438525453038
11.082115952089795
10.636127261040281
10.711759664455933
12.457121032641474
12.978326610668605
14.624584343805381
KL Divergence Results:
{'seq_temporal_HUFL': 11.15438525453038, 'seq_temporal_HULL': 11.082115952089795, 'seq_temporal_LUFL': 10.636127261040281, 'seq_temporal_LULL': 10.711759664455933, 'seq_temporal_MUFL': 12.457121032641474, 'seq_temporal_MULL': 12.978326610668605, 'seq_temporal_OT': 14.624584343805381}


# LSTM downstream

In [ ]:
real_data = loader.dataframe()

# 2. Drop the unwanted column
real_data = real_data.drop(columns=["seq_id", "seq_time_id", "seq_out_outcome"], errors="ignore")
df_synth = df_synth.drop(columns=["seq_id", "seq_time_id", "seq_out_outcome"], errors="ignore")

In [ ]:
#@title ✧.* libraries ✧.*

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler

recheck LSTM notebook for

In [ ]:

# ──────── Load synthetic training data ────────

df_synth = pd.read_csv("/content/drive/Shareddrives/sp_env/synthetic_datasets/TimeGAN/electricity/041725-104510-timeganplugin-n_3000.csv")
scaler = MinMaxScaler()
data_synth_scaled = scaler.fit_transform(df_synth.values)
data_synth = torch.tensor(data_synth_scaled, dtype=torch.float32)

# ──────── Load real held-out test data (same columns, same scale) ────────
df_real_test = real_data
data_real_scaled = scaler.transform(df_real_test.values)  # Use SAME scaler
data_real = torch.tensor(data_real_scaled, dtype=torch.float32)

# ──────── Sequence builder ───────────
def make_sequences(data, seq_len):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return torch.stack(X), torch.stack(y)

SEQ_LEN = sequence_length

# Sequences for synthetic (train)
X_train, y_train = make_sequences(data_synth, SEQ_LEN)
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)

# Sequences for real (test)
X_test, y_test = make_sequences(data_real, SEQ_LEN)


In [ ]:
#@title ✧.* model definition and training ✧.*

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, mean_squared_error

# ─── Model Definition ──────────────────────────────────────
class ShallowLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=32):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        _, (hn, _) = self.lstm(x)  # hn shape: (1, batch, hidden_size)
        out = self.linear(hn.squeeze(0))  # squeeze to (batch, hidden_size)
        return out


"""
Based on a study by Deswal and Kumar(2024), they indicated the following
hyperparameters to be the best case for the dataset in their study (stock price).

epoch = 200
batch_size = 32
neurons = 512
learning_rate = 0.002
dropout = 0.01

*did not use yet due to long training time
"""

class TunedLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=64, dropout=0.01):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, dropout=dropout, batch_first=True)
        self.linear = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        output, _ = self.lstm(x)
        last = output[:, -1, :]  # take the last time step
        return self.linear(last)

# ─── Model Init ─────────────────────────────────────────────
# model = TunedLSTM(input_size=X_train.shape[2], hidden_size=64, dropout=0.01)
model = ShallowLSTM(input_size=X_train.shape[2], hidden_size=32)

# ─── Optimizer & Loss ───────────────────────────────────────
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

# ─── Training ───────────────────────────────────────────────
EPOCHS = 50
for epoch in range(1, EPOCHS + 1):
    model.train()
    for xb, yb in train_loader:
        pred = model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # if epoch % 10 == 0 or epoch == 1:
    print(f"Epoch {epoch}: Train MSE = {loss.item():.6f}")